In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('april2021.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.drop_duplicates(subset=["link"], inplace=True)
df.set_index(['author', 'link'], inplace=True)
df.head()

date  \
author               link                                                               
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  1.615520e+09   
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  1.616182e+09   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  1.616898e+09   
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...  1.617134e+09   
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...  1.616439e+09   

                                                                                                                      post  \
author               link                                                                                                    
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  **EDIT: Getting a lot of messages and question...   
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  [Timestamp!](https://imgur.com/a/2f78aSe)\n\nH...   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  [Timestamp](https://imgur.com/a/OtOkI8s)   \n\...   
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...  Heya! [Timestamp](https://i.imgur.com/A28F0cL....   
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...  ~~Heya!~~ [~~Timestamp~~](https://i.imgur.com/...   

                                                                                                                   rawtext  \
author               link                                                                                                    
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  sold built switch couture alice and gmk hennes...   
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  sold built switch couture alice and gmk hennes...   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  bought gmk handarbeit from /u/iovercallhistioc...   
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...   sold gmk copper bundle to u/affectionate-plum193   
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...   sold gmk copper bundle to u/affectionate-plum193   

                                                                                                                     title  
author               link                                                                                                   
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  [US-CA] [H] HHKB Classic, PC Leaf60 HHKB, GMK ...  
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  [US-CA] [H] Modern M0110, KBDfans D60, GMK Hen...  
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  [US-TX] [H] GMK Metropolis sealed, Handarbeit ...  
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...  [EU-NL] [H] GMK Yuri, GMK honey, ePBT Ivory, E...  
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...  [EU-NL] [H] Ultimate GMK Copper bundle [W] PayPal

In [3]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()
only_gmk.shape, df.shape

((579, 6), (811, 6))

In [5]:
def split_on_gmk(x):
    a = x.lower().split("gmk")[1]
    return "gmk " + a.split()[0]

only_gmk['product_name'] = only_gmk['rawtext'].apply(split_on_gmk)
only_gmk

date  \
author               link                                                               
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  1.615520e+09   
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  1.616182e+09   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  1.616898e+09   
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...  1.617134e+09   
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...  1.616439e+09   
...                                                                               ...   
juicetinmk           /r/mechmarket/comments/mm9sqs/usca_h_gmk_night_...  1.617823e+09   
kiwi_kal             /r/mechmarket/comments/mvtcvb/ustx_h_gmk_pulse_...  1.619051e+09   
BraveKeyboards       /r/mechmarket/comments/mtnvun/usma_h_mizu_artis...  1.618788e+09   
rapierpoint          /r/mechmarket/comments/miy8do/usnyh_custom_hhkb...  1.617412e+09   
MeteoricJello        /r/mechmarket/comments/my851u/hkh_gmk_honey_alp...  1.619357e+09   

                                                                                                                      post  \
author               link                                                                                                    
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  **EDIT: Getting a lot of messages and question...   
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  [Timestamp!](https://imgur.com/a/2f78aSe)\n\nH...   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  [Timestamp](https://imgur.com/a/OtOkI8s)   \n\...   
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...  Heya! [Timestamp](https://i.imgur.com/A28F0cL....   
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...  ~~Heya!~~ [~~Timestamp~~](https://i.imgur.com/...   
...                                                                                                                    ...   
juicetinmk           /r/mechmarket/comments/mm9sqs/usca_h_gmk_night_...  Hi all, I have a few things for sale in the Co...   
kiwi_kal             /r/mechmarket/comments/mvtcvb/ustx_h_gmk_pulse_...  Timestamp: [https://i.imgur.com/9WpduzC.jpg](h...   
BraveKeyboards       /r/mechmarket/comments/mtnvun/usma_h_mizu_artis...  [Timestamp](https://imgur.com/gallery/tsgXrAz)...   
rapierpoint          /r/mechmarket/comments/miy8do/usnyh_custom_hhkb...  [Timestamp](https://imgur.com/a/jzReoEM)\n\nHe...   
MeteoricJello        /r/mechmarket/comments/my851u/hkh_gmk_honey_alp...  [Timestamp](https://imgur.com/a/JlIZUfy)\n\n&#...   

                                                                                                                   rawtext  \
author               link                                                                                                    
ohwhatitsmeels       /r/mechmarket/comments/m38spq/usca_h_hhkb_class...  sold built switch couture alice and gmk hennes...   
                     /r/mechmarket/comments/m8p437/usca_h_modern_m01...  sold built switch couture alice and gmk hennes...   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/mese1t/ustx_h_gmk_metrop...  bought gmk handarbeit from /u/iovercallhistioc...   
Ayylmao69420xd       /r/mechmarket/comments/mgofvm/eunl_h_gmk_yuri_g...   sold gmk copper bundle to u/affectionate-plum193   
                     /r/mechmarket/comments/mav2om/eunl_h_ultimate_g...   sold gmk copper bundle to u/affectionate-plum193   
...                                                                                                                    ...   
juicetinmk           /r/mechmarket/comments/mm9sqs/usca_h_gmk_night_...             sold gmk finer things to /u/hiromitsu6   
kiwi_kal             /r/mechmarket/comments/mvtcvb/ustx_h_gmk_pulse_...           bought gmk pulse mitolet from u/kiwi_kal   
BraveKeyboards       /r/

In [21]:
from collections import defaultdict
all_prices = defaultdict(list)

money_regex = r'\$(\d+)|(\d+)\$'

sets = ['base', 'nov', 'alphas', 'accents', 'bar', 'light base', 'dark base', 'deskmat', 'rama']

def match_product(df):
    for row in df.itertuples():
        s = row.post_lower.split("\n")
        product = row.product_name.lower()
        #print(row[0][1])
        for l in s:
            low = l.lower()
            if "gmk " in low and ('~~' in low or 'sold' in low):
                a = low.split("gmk ")[1]
                product_name = "gmk " + re.split(r" |~", a)[0]
                
                matches = re.split(money_regex, low)
                temp_data = {}
                if len(matches) > 1:
                    if "red" in product:
                        print(low,"\n")
                    #print(f"FULL LINE: {low}")
                    for i in range(0, len(matches)-1, 3):
                        curr_price = int(matches[i+1] if matches[i+1] else matches[i+2])
                        curr_str = matches[i]
                        kits = []
                        removeBase = False
                        empty=True
                        for x in sets:
                            if x in curr_str:
                                empty=False
                                if x == 'nov':
                                    kits.append('novelties')
                                elif x == 'light base' or x == 'dark base':
                                    kits.insert(0, f'{x}')
                                    removeBase = True
                                else:
                                    kits.append(x)
                        products = product_name
                        if i == 0 and not kits:
                            kits = ['base']
                        for x in kits:
                            if removeBase and x == 'base':
                                continue
                            products += f', {x}'
                        
                        if i == 0:
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price

                            if "red" in product:
                                print(products)
                                print(temp_data)
                                print()
                        
                        if kits and i > 0:
                            #print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                            all_prices[temp_data['products']].append(temp_data['price'])
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        else:
                            temp_data["price"] = min(temp_data["price"], curr_price)
                        
                    #print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                    all_prices[temp_data['products']].append(temp_data['price'])

count = 1  
for i, new_df in only_gmk.groupby(level=0):
    match_product(new_df)
    count -= 1
    

print(all_prices)

|drop + redsuns gmk red samurai 65%|bought on mm, used one time and i don't really like abs|~~$70 shipped~~ **sold**| 

gmk red, base
{'products': 'gmk red, base', 'str': "|drop + redsuns gmk red samurai 65%|bought on mm, used one time and i don't really like abs|~~", 'price': 70}

* ~~gmk mitolet - lightly used in bags $115 shipped~~ **sold for asking price** 

gmk mitolet, base
{'products': 'gmk mitolet, base', 'str': '* ~~gmk mitolet - lightly used in bags ', 'price': 115}

* ~~gmk red samurai tkl kit used in bags - $75 shipped~~ **sold for asking price** 

gmk red, base
{'products': 'gmk red, base', 'str': '* ~~gmk red samurai tkl kit used in bags - ', 'price': 75}

|~~gmk metaverse r1 base kit + r2 novelties~~|~~base kit has a lot of shine and missing a black f2 key and a right 1.25u alt gr key. keys were missing from previous seller. novelties are practically brand new never typed on.~~|~~$250 + shipping. (add $15 for shipping in jtk polycarb trays)~~ sold| 

gmk metaverse, base,